# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it.

You will find more info about virtual environments in the course content and on the web.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [242]:
# import the requests library (1 line)
import requests

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = "https://country-leaders.onrender.com"

# assign the /status endpoint to another variable called status_url (1 line)
status_url = f"{root_url}/status"

# query the /status endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(status_url)

# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code == 200:
    print(req.text)
else:
    print(f"Status Code: {req.status_code}")

"Alive"


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [243]:
# Set the countries_url variable (1 line)
countries_url = f"{root_url}/countries"

# query the /countries endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(countries_url)

# Get the JSON content and store it in the countries variable (1 line)
countries = req.json()

# display the request's status code and the countries variable (1 line)
print(req.status_code, countries)

403 {'message': 'The cookie is missing'}


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [244]:
# Set the cookie_url variable (1 line)
cookie_url = f"{root_url}/cookie"

# Query the enpoint, set the cookies variable and display it (2 lines)
req = requests.get(cookie_url)
cookies = req.cookies
print(cookies)

<RequestsCookieJar[<Cookie user_cookie=eaf302f5-5a6e-4459-91e0-aee34cca0b91 for country-leaders.onrender.com/>]>


Try to query the countries endpoint using the cookie, save the output and print it.

In [245]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries = requests.get(countries_url, cookies=cookies).json()

# display the countries variable (1 line)
print(countries)

['us', 'be', 'ma', 'ru', 'fr']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [246]:
'''
# Set the leaders_url variable
leaders_url = f"{root_url}/leaders"

# query the /leaders endpoint, assign the output to the leaders variable (refresh cookie first if necessary)
leaders = requests.get(leaders_url, cookies=cookies, params={"country": first_country}).json()

# display the leaders variable
print(leaders)
'''

'\n# Set the leaders_url variable\nleaders_url = f"{root_url}/leaders"\n\n# query the /leaders endpoint, assign the output to the leaders variable (refresh cookie first if necessary)\nleaders = requests.get(leaders_url, cookies=cookies, params={"country": first_country}).json()\n\n# display the leaders variable\nprint(leaders)\n'

In [247]:
# Refresh the cookie by querying the /cookie endpoint
req = requests.get(cookie_url)
cookies = req.cookies  # Update the cookies variable with the new cookie
print("New cookie obtained:", cookies)

# Query the /countries endpoint using the refreshed cookie
countries = requests.get(countries_url, cookies=cookies).json()
print("Countries:", countries)

# Set the leaders_url variable
leaders_url = f"{root_url}/leaders"

# Query the /leaders endpoint using the refreshed cookie and parameters (using the first country as an example)
first_country = countries[0] if isinstance(countries, list) else list(countries.keys())[0]
leaders = requests.get(leaders_url, cookies=cookies, params={"country": first_country}).json()

# Display the leaders variable
print("Leaders:", leaders)


New cookie obtained: <RequestsCookieJar[<Cookie user_cookie=57865033-3031-43e6-82da-8f8167e52951 for country-leaders.onrender.com/>]>
Countries: ['us', 'be', 'ma', 'ru', 'fr']
Leaders: [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincol

In [248]:
'''
# Set the leaders_url variable (1 line)
leaders_url = f"{root_url}/leaders"

# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders = requests.get(leaders_url, cookies=cookies).json()

# display the leaders variable (1 line)
print(leaders)
'''

'\n# Set the leaders_url variable (1 line)\nleaders_url = f"{root_url}/leaders"\n\n# query the /leaders endpoint, assign the output to the leaders variable (1 line)\nleaders = requests.get(leaders_url, cookies=cookies).json()\n\n# display the leaders variable (1 line)\nprint(leaders)\n'

It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [249]:
# first_country = list(countries.keys())[0]
# query the /leaders endpoint using cookies and parameters (take any country in countries)
# assign the output to the leaders variable (1 line)
leaders = requests.get(leaders_url, cookies=cookies, params={"country": first_country}).json()

# display the leaders variable (1 line)
print(leaders)

[{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'place_of_bi

### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [250]:
# 4 lines
# Initialize an empty dictionary to store leaders data for each country
leaders_per_country = {}

# Loop through each country in the countries list and query the /leaders endpoint
for country in countries:
    leaders_per_country[country] = requests.get(leaders_url, cookies=cookies, params={"country": country}).json()

# Display the dictionary with leaders data for each country
print(leaders_per_country)


{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'plac

In [251]:
# or 1 line
leaders_per_country = {country: requests.get(leaders_url, cookies=requests.get(cookie_url).cookies, params={"country": country}).json() for country in countries}


It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [252]:
# < 15 lines
def get_leaders():
    # Define the URLs
    root_url = "https://country-leaders.onrender.com"
    leaders_url = f"{root_url}/leaders"
    cookie_url = f"{root_url}/cookie"
    countries_url = f"{root_url}/countries"
    
    # Get the cookies
    cookies = requests.get(cookie_url).cookies
    
    # Get the countries
    countries = requests.get(countries_url, cookies=cookies).json()
    
    # Initialize dictionary to store leaders data
    leaders_per_country = {
        country: requests.get(leaders_url, cookies=cookies, params={"country": country}).json()
        for country in countries
    }
    
    # Return the dictionary
    return leaders_per_country


Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [253]:
# 2 lines
# Test the get_leaders function and save the result in leaders_per_country
leaders_per_country = get_leaders()

# Display the leaders_per_country dictionary
print(leaders_per_country)

{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'plac

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [254]:
print(leaders_per_country['us'])

[{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'place_of_bi

In [255]:
# Print the details of the first leader for the 'us' country
print(leaders_per_country['us'][0])  # Assuming it's a list of leaders


{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}


In [256]:
print(leaders_per_country['be'][0]['wikipedia_url'])

https://nl.wikipedia.org/wiki/Guy_Verhofstadt


In [257]:
leaders_per_country['ma'][1]['wikipedia_url']

'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF'

In [258]:
# Get the Wikipedia URL of the first leader (example)
wiki_url = leaders_per_country['ma'][1]['wikipedia_url']  # Access the 'wikipedia_url' key

# Query the Wikipedia page
response = requests.get(wiki_url)

# Display the text of the page
print(response.text)


<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" lang="ar" dir="rtl">
<head>
<meta charset="UTF-8">
<title>الحسن الثاني بن محمد - ويكيبيديا</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 ve

Explanation:  
leaders_per_country['us'] is a list containing leader dictionaries.  
leaders_per_country['us'][0] accesses the first leader in that list.  
leaders_per_country['us'][0]['wikipedia_url'] retrieves the Wikipedia URL of that leader.  

In [259]:
'''
Doesn`t work
# 3 lines
# Get the Wikipedia URL of the first leader (example)
wiki_url = leaders_per_country['us']['id'][1]['wikipedia']  # Adjust keys as necessary

# Query the Wikipedia page
response = requests.get(wiki_url)

# Display the text of the page
print(response.text)
'''

"\nDoesn`t work\n# 3 lines\n# Get the Wikipedia URL of the first leader (example)\nwiki_url = leaders_per_country['us']['id'][1]['wikipedia']  # Adjust keys as necessary\n\n# Query the Wikipedia page\nresponse = requests.get(wiki_url)\n\n# Display the text of the page\nprint(response.text)\n"

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [260]:
# 3 lines
from bs4 import BeautifulSoup  # Import the BeautifulSoup library

soup = BeautifulSoup(response.text, 'html.parser')  # Parse the raw HTML text

print(soup.prettify())  # Print the prettified HTML for inspection

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" dir="rtl" lang="ar">
 <head>
  <meta charset="utf-8"/>
  <title>
   الحسن الثاني بن محمد - ويكيبيديا
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-wi

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [261]:
# Find all <p> tags (paragraphs) in the HTML
paragraphs = soup.find_all('p')

# Find the first non-empty paragraph and print its text
print(next(paragraph.get_text() for paragraph in paragraphs if paragraph.get_text().strip()))


الحَسَن الثاني بِنْ مُحمد بِنْ يوسف العَلوي (9 يوليو 1929 – 23 يوليو 1999)،[12] ثاني ملوك المملكة المغربية بعد الإستقلال، والملك الثاني والعشرين للمغرب من سلالة العلويين الفيلاليين، تولى حكم المملكة المغربية خلفًا لوالده الملك محمد الخامس في 26 فبراير 1961 وحتى وفاته في 23 يوليو 1999. ينتمي الملك الحسن الثاني إلى السلالة العلوية التي تعود في نسبها إلى الحسن بن علي بن أبي طالب، وتحكم المغرب منذ عام 1666 ميلادية ويلقب الحاكم منهم بأمير المؤمنين.



In [262]:
'''
# Find all <p> tags (paragraphs) in the HTML
paragraphs = soup.find_all('p')

# Find the first non-empty paragraph and print its text
for paragraph in paragraphs:
    if paragraph.get_text().strip():  # Skip empty paragraphs
        print(paragraph.get_text())
        break  # Stop once we find the first meaningful paragraph
'''

"\n# Find all <p> tags (paragraphs) in the HTML\nparagraphs = soup.find_all('p')\n\n# Find the first non-empty paragraph and print its text\nfor paragraph in paragraphs:\n    if paragraph.get_text().strip():  # Skip empty paragraphs\n        print(paragraph.get_text())\n        break  # Stop once we find the first meaningful paragraph\n"

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [263]:
# more than 10 lines
def get_first_paragraph(wikipedia_url, keywords=None):
    print(wikipedia_url)  # For tracking URLs
    
    response = requests.get(wikipedia_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.find_all('p')

    # Iterate through paragraphs and look for keyword matches or longer paragraphs
    for paragraph in paragraphs:
        text = paragraph.get_text().strip()
        if keywords:
            # Check if any keyword is in the text
            if any(keyword.lower() in text.lower() for keyword in keywords):
                return text
        elif len(text) > 50:  # A fallback to get a decently long paragraph
            return text
    return None  # Return None if no paragraph is found


In [264]:
# Define URLs with additional countries
urls = {
    'us': 'https://en.wikipedia.org/wiki/George_Washington',
    'fr': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy',
    'ma': 'https://ar.wikipedia.org/wiki/الحسن_الثاني_بن_محمد',
    'ru': 'https://ru.wikipedia.org/wiki/Медведев,_Дмитрий_Анатольевич',
    'be': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt'  # Dutch page for Belgium leader
}

# Define keywords for each country in their respective languages
keywords = {
    'us': ['President', 'United States'],
    'fr': ['président', 'France'],
    'ma': ['ملك', 'المغرب'],  # Arabic for "King" and "Morocco"
    'ru': ['президент', 'Россия'],  # Russian for "President" and "Russia"
    'be': ['Premier', 'België']  # Dutch for "Premier" and "Belgium"
}

# Loop over each URL, passing keywords if available
for code, url in urls.items():
    kw = keywords.get(code)  # Retrieve keywords if they exist, or None if not
    print(f"Extracted paragraph for {code.upper()}:")
    print(get_first_paragraph(url, kw))  # Call function with or without keywords
    print('-' * 40)


Extracted paragraph for US:
https://en.wikipedia.org/wiki/George_Washington
George Washington (February 22, 1732 – December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country".
----------------------------------------
Extracted paragraph for FR:
https://fr.wikipedia.org/wiki/Nicolas_Sarkozy
Nicolas Sarközy de Nagy-Bocsa[d], dit Nicolas Sarkozy (/ni.kɔ.la saʁ.kɔ.zi/[e] ; Écouter), né le 28 janvier 1955 à Paris (France), est un homme d'État français. Il est président de la République française du 16 mai 2007 au 15 mai 20

At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [268]:
from bs4 import BeautifulSoup
import requests

def get_first_paragraph(wikipedia_url):
    print(wikipedia_url)  # Keep this for the rest of the notebook
    
    # Request the Wikipedia page
    response = requests.get(wikipedia_url)
    
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all <p> tags (paragraphs)
    paragraphs = soup.find_all('p')
    
    # Loop through the paragraphs and find the first relevant one
    for paragraph in paragraphs:
        text = paragraph.get_text().strip()
        if 'George Washington' in text:  # Replace with a reliable string
            first_paragraph = text
            break
    
    return first_paragraph


In [267]:
wikipedia_url = 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt'  # Example URL
first_paragraph = get_first_paragraph(wikipedia_url)  # Call the function
print(first_paragraph)  # Display the result

https://nl.wikipedia.org/wiki/Guy_Verhofstadt


UnboundLocalError: cannot access local variable 'first_paragraph' where it is not associated with a value

### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [ ]:
# 3 lines
import re

# Clean the extracted paragraph using regex
cleaned_text = re.sub(r'\[.*?\]|\<.*?\>|\(.*?\)', '', first_paragraph)

print(cleaned_text)



иностранные:


In [ ]:
# more lines
import re

def clean_text(paragraph):
    # Remove citations in square brackets, like [1], [citation needed]
    text_no_brackets = re.sub(r'\[.*?\]', '', paragraph)
    
    # Remove HTML tags if there are any, such as <i>, <b>, etc.
    text_no_html = re.sub(r'<.*?>', '', text_no_brackets)
    
    # Remove text within parentheses, which is often pronunciation or additional notes
    text_no_parentheses = re.sub(r'\(.*?\)', '', text_no_html)
    
    # Remove multiple spaces, tabs, or newline characters for cleaner output
    cleaned_text = re.sub(r'\s+', ' ', text_no_parentheses).strip()
    
    return cleaned_text



Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [ ]:
# 10 lines


Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [ ]:
# < 20 lines


## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [ ]:
# < 20 lines

In [ ]:
# Check the output of your function (2 lines)


Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [ ]:
# < 25 lines



Check the output of your function again.

In [ ]:
# Check the output of your function (1 line)


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [ ]:
# < 20 lines


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [ ]:
# <25 lines


Test your new functions.



## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [ ]:
# 3 lines


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [ ]:
# 3 lines


Make a function `save(leaders_per_country)` to call this code easily.

In [ ]:
# 3 lines


In [ ]:
# Call the function (1 line)


## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!